In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import (accuracy_score, mean_absolute_percentage_error)
from sklearn.model_selection import (GridSearchCV, StratifiedKFold,
                                     cross_val_score)
from sklearn.ensemble import (BaggingClassifier, BaggingRegressor,
                              RandomForestClassifier, RandomForestRegressor)
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, train_test_split

In [2]:
#df = pd.read_csv("observation_202403041348.csv")
#
##df.head()
#df = df[df["operators_avg_service_time_last_month"] > 0]
#df = df[df["operators_avg_service_time_last_hour"] > 0]
#
#y = df["wait_duration_class_2"]
#df.drop(["wait_duration", "wait_duration_class_1", "wait_duration_class_2", "id"], axis=1, inplace=True)
#X = df
#
## Initialize a stratified split of our dataset for the validation process
#skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#
## Initialize the classifier with the default parameters 
#rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
#
## Train it on the training set
#results = cross_val_score(rfc, X, y, cv=skf)
#
## Evaluate the accuracy on the test set
#print("CV accuracy score: {:.2f}%".format(results.mean() * 100))

CV accuracy score: 80.49%


In [3]:

df_reg = pd.read_csv("observation_202403041348.csv")
df_reg["month"] = df_reg["month"].astype("object")
df_reg["day_of_month"] = df_reg["day_of_month"].astype("object")
df_reg["day_of_week"] = df_reg["day_of_week"].astype("object")
df_reg["service_id"] = df_reg["service_id"].astype("object")
df_reg["service_operators_set"] = df_reg["service_operators_set"].astype("object")
df_reg["is_urgent"] = df_reg["is_urgent"].astype("object")
df_reg["weather_type"] = df_reg["weather_type"].astype("object")

df_reg = df_reg[df_reg["operators_avg_service_time_last_month"] > 0]
df_reg = df_reg[df_reg["operators_avg_service_time_last_hour"] > 0]
#df_reg = df_reg[df_reg["wait_duration"] > 0]

y = df_reg["wait_duration"]
df_reg.drop(["wait_duration", "wait_duration_class_1", "wait_duration_class_2", "id"], axis=1, inplace=True)

X_train, X_holdout, y_train, y_holdout = train_test_split(
    df_reg.values, y, test_size=0.3, random_state=17, shuffle=True
)

# Random Forest
rf = RandomForestRegressor(n_estimators=10).fit(X_train, y_train)
rf_predict = rf.predict(X_holdout)

mse = mean_squared_error(y_holdout, rf_predict)
relative_error = np.sqrt(mse) / np.mean(y_holdout.array)
#relative_error_2 = (np.absolute(rf_predict - y_holdout) / y_holdout).sum() / y_holdout.shape[0]
mape = mean_absolute_percentage_error(y_holdout, rf_predict)
print("Relative error value:", relative_error)
print("mape:", mape)

grouped_relative_errors = []

minute = 0
while minute < 90:
    group_indices = np.where((minute * 60 < y_holdout) & (y_holdout <= (minute + 1) * 60))[0]
    minute = minute + 1
    group_X_holdout = X_holdout[group_indices]
    group_y_holdout = y_holdout.array[group_indices]
    if group_X_holdout.shape[0] < 1:
        continue
    group_y_pred = rf.predict(group_X_holdout)
    group_mse = mean_squared_error(group_y_holdout, group_y_pred)
    group_relative_error = np.sqrt(group_mse) / np.mean(group_y_holdout)
    #group_relative_error_2 = (np.absolute(group_y_pred - group_y_holdout) / group_y_holdout).sum() / group_y_holdout.shape[0]
    mape = mean_absolute_percentage_error(group_y_holdout, group_y_pred)
    grouped_relative_errors.append((minute, group_relative_error, mape))

print("Relative error value grouped by target value:")
for target_value, relative_error, mape in grouped_relative_errors:
    print(f"Target value: {target_value}, Relative error: {relative_error}, mape: {mape}")


Relative error value: 1.283145332750591
mape: 3542077932641987.0
Relative error value grouped by target value:
Target value: 1, Relative error: 8.952844634304235, mape: 6.985055975748926
Target value: 2, Relative error: 2.0934044106451744, mape: 0.925170921869014
Target value: 3, Relative error: 2.7055138101153546, mape: 0.6723806388441558
Target value: 4, Relative error: 0.6762333902437047, mape: 0.5117623328223275
Target value: 5, Relative error: 0.5657992160477741, mape: 0.4355459224853793
Target value: 6, Relative error: 0.49438245147913773, mape: 0.3791636330547834
Target value: 7, Relative error: 0.43500826784162877, mape: 0.33481564369410427
Target value: 8, Relative error: 0.39314134958163083, mape: 0.30697378183561813
Target value: 9, Relative error: 0.36925986547359757, mape: 0.2915693722195559
Target value: 10, Relative error: 0.4996023812398667, mape: 0.27936979581014487
Target value: 11, Relative error: 0.3270187050300054, mape: 0.25500588923337575
Target value: 12, Relati

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169280 entries, 0 to 169279
Data columns (total 25 columns):
 #   Column                                 Non-Null Count   Dtype  
---  ------                                 --------------   -----  
 0   id                                     169280 non-null  int64  
 1   month                                  169280 non-null  int64  
 2   day_of_month                           169280 non-null  int64  
 3   day_of_week                            169280 non-null  int64  
 4   hour                                   169280 non-null  int64  
 5   minute                                 169280 non-null  int64  
 6   non_working_days_before                169280 non-null  int64  
 7   non_working_days_after                 169280 non-null  int64  
 8   service_id                             169280 non-null  int64  
 9   minutes_from_opening                   169280 non-null  int64  
 10  minutes_to_closing                     169280 non-null  